In [1]:
import os
import sys
import tensorflow as tf
import utils
from time import time
import models as Models

NUM_EPOCHS = 1
BATCH_SIZE = 500
SEQ_LEN = 500
VOCAB_SIZE = 500

def run(model, sess, dataset):
    avg_correct = 0.0
    for x, masks, y in dataset:
        
        input_feed = {
            model.inputs: utils.convert_to_np(x),
            model.masks: utils.convert_to_np(masks),
            model.labels: utils.convert_to_np(y),
        }
        
        output_feed = [model.accuracy, model.op]
        num_correct, _ = sess.run(output_feed, input_feed)
        avg_correct += num_correct
    avg_correct /= len(dataset)
    return avg_correct

def train(vocab, embeddings, train_data, test_data):
    model = Models.DenseQRNNModel(embeddings, BATCH_SIZE, SEQ_LEN, VOCAB_SIZE)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        epoch = model.epoch.eval()
        
        for i in range(epoch, NUM_EPOCHS):
            train_acc = run(model, sess, train_data)
            print ('epoch', i, 'average train acc:', train_acc)
            sess.run(tf.assign(model.epoch, i+1))
        
        test_acc = run(model, sess, test_data)
        print ('average test acc:', test_acc)

def init_embeddings(embeds, vocab, dim):
    embed_list = []
    for i in range(3):
        var = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[dim]), dtype=tf.float32)
        embed_list.append(var)
    for _id, word in vocab.items():
        if int(_id) in embeds.keys():
            embed_list.append(tf.constant(embeds[_id], dtype=tf.float32))
        else:
            embed_list.append(embed_list[2])
    return tf.stack(embed_list, axis=0)

if __name__ == '__main__':
    trainset, test, vocab = utils.get_datasets(batch_size=BATCH_SIZE,num_words=VOCAB_SIZE,seq_len=SEQ_LEN)
    embeds = utils.get_embeddings(vocab, './glove.6B/glove.6B.300d.txt')
    embeds = init_embeddings(embeds, vocab, 300)
    train(vocab, embeds, trainset, test)

Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
epoch 0 average train acc: 0.49796001851558686
average test acc: 0.4959200215339661
